 # Cours: Applied Data Science Capstone
## Section 0 : 
### The section 0 of this notebook is created for Capstone_Coursera project as just 
### the first test of notebooke creation and check sharing in GiyHub
### Other sections will be following the cours amendes
 

In [89]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


 # Cours: Applied Data Science Capstone
## section 1: 
### Segmenting and Clustering Neighborhoods in Toronto


### section 1-1
#### Transfer data of Toronto's  neighborhoods  from wikipedia to a dataframe

In [90]:
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, attrs={"class": "wikitable"})[0] # 0 is for the 1st table in this particular page
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Section 1-2
#### Ignore cells with a borough that is Not assigned

In [91]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Section 1-3
#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [92]:
print ("this section is not applicable as the latest version \n of wikipedia is in the same format as we need.")

this section is not applicable as the latest version 
 of wikipedia is in the same format as we need.


### Section 1-4:
#### 1-4-1 If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
#### 1-4-2 Reset index

In [97]:
df['Neighbourhood'] = np.where(df['Neighbourhood']=='Not assigned', df['Borough'], df['Neighbourhood'])
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Section 1-4:
#### Shape of dataframe


In [98]:
df.shape

(103, 3)